# Lab1 — PyTorch Foundations for Computer Vision

**Course**: Deep Learning for Image Analysis

**Class**: M2 IASD App  

**Professor**: Mehyar MLAWEH

---

## Objectives
By the end of this lab, you should be able to:

- Understand how **neurons and layers** are implemented in PyTorch
- Manipulate **tensors** and reason about shapes
- Use **autograd** to compute gradients
- Implement a **training loop** yourself
- Connect theory (neurons, loss, backprop) to actual code

⚠️ This notebook is **intentionally incomplete**.  
Whenever you see **`# TODO`**, you are expected to write code.


**Deadline:** 🗓️ **Saturday, February 7th (23:59)**

## 🤖 A small (honest) note before you start

Let’s be real for a second.

 I know you **can use LLMs (ChatGPT, Copilot, Claude, etc.)** to help you with this lab.  
And yes, **I use them too**, so don’t worry 😄

👉 **You are allowed to use AI tools.**  
But here’s the deal:

- Don’t just **copy–paste** code you don’t understand  
- Take time to **read, question, and modify** what the model gives you  
- If you can solve a block **by yourself, without AI**, that’s excellent

Remember:

> AI can write code for you, but **only you can understand it** — and understanding is what matters for exams, projects, and real work.

Use these tools **as assistants, not as replacements for thinking**.

---

## 📚 Useful documentation (highly recommended)

You will often find answers faster (and more reliably) by checking the official documentation:

- **PyTorch main documentation**  
  https://pytorch.org/docs/stable/index.html

- **PyTorch tensors**  
  https://pytorch.org/docs/stable/tensors.html

- **Neural network modules (`torch.nn`)**  
  https://pytorch.org/docs/stable/nn.html

- **Loss functions** (`BCEWithLogitsLoss`, CrossEntropy, etc.)  
  https://pytorch.org/docs/stable/nn.html#loss-functions

- **Optimizers** (`SGD`, `Adam`, …)  
  https://pytorch.org/docs/stable/optim.html

If you learn how to **navigate the documentation**, you are already thinking like a real AI engineer 👌

---

## PART I

## 0) Colab setup — GPU check

**Instructions**
1. In Colab: `Runtime → Change runtime type to GPU T4`
2. Select **GPU** (T4 in Colab)
3. Save and restart runtime

Then run the cell below.


In [1]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

# TODO: set the device correctly (cuda if available, else cpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)


PyTorch version: 2.9.0+cu126
CUDA available: True
Using device: cuda


## 1) Imports and reproducibility


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# TODO: fix the random seed for reproducibility
torch.manual_seed(42)


## 2) PyTorch tensors and shapes

Tensors are multi-dimensional arrays that support:
- GPU acceleration
- automatic differentiation

Understanding **shapes** is critical in deep learning.


In [3]:
# Examples
a = torch.tensor([1.0, 2.0, 3.0])
b = torch.randn(4, 5)

print("a shape:", a.shape)
print("b shape:", b.shape)


a shape: torch.Size([3])
b shape: torch.Size([4, 5])


### 🔍 Question (answer inside the markdown)
- How many dimensions does tensor `b` have?

b is a tensor of dimension 2.



- What does each dimension represent conceptually?
When working with tensors of size 2, usually the first dimension represents the size of the dataset, the second one is the number of features for each observation.


### ✅Tensor operations

Complete the following:

1. Create a tensor `x` of shape `(8, 3)` with random values  
2. Compute:
   - the **mean of each column**
   - the **L2 norm of each row**
3. Normalize `x` **row-wise** using the L2 norm

In [4]:
# TODO: create x
x = torch.randn(8, 3)

# TODO: column mean
col_mean = torch.mean(x, dim=0)

# TODO: row-wise L2 norm
row_norm = torch.norm(x, p=2, dim=1)

# TODO: normalized tensor
x_normalized = x / row_norm.view(-1, 1)

print(x.shape, col_mean.shape, row_norm.shape, x_normalized.shape)

print(x, x_normalized)
print(torch.norm(x_normalized, p=2, dim=1))


torch.Size([8, 3]) torch.Size([3]) torch.Size([8]) torch.Size([8, 3])
tensor([[ 0.3189, -0.4245,  0.3057],
        [-0.7746,  0.0349,  0.3211],
        [ 1.5736, -0.8455,  0.3672],
        [ 0.1754,  1.3852, -0.4459],
        [ 1.4451,  0.8564,  2.2181],
        [ 0.5232,  1.1754,  0.5612],
        [-0.4527, -0.7718, -0.1722],
        [ 0.5238,  0.0566,  0.4263]]) tensor([[ 0.5205, -0.6929,  0.4990],
        [-0.9230,  0.0416,  0.3826],
        [ 0.8629, -0.4636,  0.2014],
        [ 0.1197,  0.9451, -0.3042],
        [ 0.5194,  0.3078,  0.7972],
        [ 0.3727,  0.8374,  0.3998],
        [-0.4969, -0.8470, -0.1890],
        [ 0.7729,  0.0835,  0.6290]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


## 3) Artificial neuron — from math to code

A neuron computes:

$$
z = \sum_i w_i x_i + b
$$

Then applies an activation function:

$$
y = g(z)
$$

This section connects directly to the theory seen in class.


In [5]:
x = torch.tensor([1.0, -2.0, 3.0])
w = torch.tensor([0.2, 0.4, -0.1])
b = torch.tensor(0.1)

z = torch.sum(x * w) + b
z


tensor(-0.8000)

### Activation functions

1. Implement **ReLU**
2. Implement **Sigmoid**
3. Apply both to `z` and compare the outputs

Which activation preserves negative values?

None of them preserves negative values. ReLU kills negative signals by setting them to 0. Sigmoid converts real values to probabilities, so to a value between 0 and 1.


In [7]:
# TODO
def relu(z):
  return torch.max(z, torch.zeros_like(z))

def sigmoid(z):
  return 1/(1+torch.exp(-z))

y_relu = relu(z)
y_sigmoid = sigmoid(z)
y_relu, y_sigmoid


(tensor(0.), tensor(0.3100))

## 4) Autograd and gradients

PyTorch uses **automatic differentiation** to compute gradients
using the **chain rule** (backpropagation).


In [8]:
x = torch.tensor([1.0, 2.0, -1.0], requires_grad=True)
w = torch.tensor([0.5, -0.3, 0.8], requires_grad=True)
b = torch.tensor(0.2, requires_grad=True)

z = torch.sum(x * w) + b
loss = (z - 1.0) ** 2

loss.backward()

print("loss:", loss.item())
print("grad w:", w.grad)
print("grad b:", b.grad)


loss: 2.890000104904175
grad w: tensor([-3.4000, -6.8000,  3.4000])
grad b: tensor(-3.4000)


### 🔍 Conceptual question

- If `b.grad > 0`, should `b` increase or decrease after a gradient descent step?
Explain **why** in one sentence.

The gradient indicates the direction of the steepest ascent. Our goal is to minimise the loss, so we take steps into the opposite direction of the gradient. Therefore, after one step of gradient DESCENT, the value of should decrease.


## 5) Toy classification dataset

We create a **linearly separable** dataset.

Label rule:
- class = 1 if `x₁ + x₂ + x₃ > 0`
- else class = 0

This mimics a very simple classification problem.


In [ ]:
# TODO: generate a dataset of size N=500 with 3 features
X = torch.randn(500, 3)
y = (torch.sum(X, dim=1) > 0).float().view(-1,1) # shape (N, 1)

# TODO: split into train (80%) and validation (20%)
train_size = 0.8*X.shape[0]
X_train = X[:int(train_size),:]
y_train = y[:int(train_size)]

X_val = X[int(train_size):,:]
y_val = y[int(train_size):]

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)


torch.Size([400, 3]) torch.Size([400, 1])
torch.Size([100, 3]) torch.Size([100, 1])


## 6) Model definition

We define a small **MLP** (fully-connected network):

`3 → 16 → 8 → 1`

Activation: ReLU  
Output: raw logits (no sigmoid)


In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            # TODO: Linear 3 → 16
            nn.Linear(3, 16),
            # TODO: ReLU
            nn.ReLU(),
            # TODO: Linear 16 → 8
            nn.Linear(16, 8),
            # TODO: ReLU
            nn.ReLU(),
            # TODO: Linear 8 → 1
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.net(x)

# TODO: create model and move it to the GPU
model = MLP().to(device)


###  parameters

1. Compute **by hand** the total number of parameters
According to https://docs.pytorch.org/docs/stable/generated/torch.nn.Linear.html, the bias term is included by default, in which case the number of parameters is:
- 3*16 + 16 biases for the first layer, which adds up to 64
- 16*8 which is 128, plus the 8 biases which is 136
- 8 weights + 1 bias for the output layer, which is 9.


In total, we thus have 209 parameters.

2. Verify your answer using PyTorch


In [ ]:
# TODO: count parameters with PyTorch
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params


209

## 7) Training loop

You must complete the full training loop:
- forward pass
- loss computation
- backward pass
- optimizer step

Loss: `BCEWithLogitsLoss`
Optimizer: `SGD`


In [ ]:
# TODO: move data to device
X_train_d = X_train.to(device)
y_train_d = y_train.to(device)
X_val_d = X_val.to(device)
y_val_d = y_val.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(1,201):
    model.train()
    optimizer.zero_grad()

    # TODO: forward
    logits = model(X_train_d)

    # TODO: loss
    loss = criterion(logits, y_train_d)

    # TODO: backward
    loss.backward()

    # TODO: update
    optimizer.step()

    if epoch % 5 == 0:
        print("Epoch", epoch, "| loss =", float(loss))


Epoch 5 | loss = 0.7348411679267883
Epoch 10 | loss = 0.7142819166183472
Epoch 15 | loss = 0.6984398365020752
Epoch 20 | loss = 0.683353841304779
Epoch 25 | loss = 0.6668954491615295
Epoch 30 | loss = 0.6465408802032471
Epoch 35 | loss = 0.6211355328559875
Epoch 40 | loss = 0.5896747708320618
Epoch 45 | loss = 0.5514328479766846
Epoch 50 | loss = 0.5064075589179993
Epoch 55 | loss = 0.4565713107585907
Epoch 60 | loss = 0.40497106313705444
Epoch 65 | loss = 0.3549319803714752
Epoch 70 | loss = 0.3094232380390167
Epoch 75 | loss = 0.2699418067932129
Epoch 80 | loss = 0.2367388904094696
Epoch 85 | loss = 0.20942749083042145
Epoch 90 | loss = 0.18705607950687408
Epoch 95 | loss = 0.16866859793663025
Epoch 100 | loss = 0.15352974832057953
Epoch 105 | loss = 0.14105993509292603
Epoch 110 | loss = 0.1306404024362564
Epoch 115 | loss = 0.12180926650762558
Epoch 120 | loss = 0.1142706647515297
Epoch 125 | loss = 0.10777018219232559
Epoch 130 | loss = 0.10210128873586655
Epoch 135 | loss = 0.097

## 8) Evaluation

1. Apply `sigmoid` to the logits
2. Convert probabilities to predictions
3. Compute **accuracy** on the validation set


In [ ]:
# TODO: evaluation
with torch.no_grad():
  logits = model(X_val_d)
  probits = torch.sigmoid(logits)
  preds =  (probits > .5).float()

print(preds.shape, y_val_d.shape)

accuracy = (preds == y_val_d).float().mean()
accuracy


torch.Size([100, 1]) torch.Size([100, 1])


tensor(0.9900, device='cuda:0')

## 9) Reflection questions (answer inside the markdown)

1. Why do we **not** apply sigmoid inside the model?
The loss function we're using in the network is BCEWithLogitsLoss. According to the doc (https://docs.pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html), this function combines Sigmoid and and BCE Loss in one layer.
2. What would happen if we removed all ReLU activations?
If we were to remove ReLU activations, there would not be any non-linearity at all. The network would only become equivalent to several matrix multiplications applied in a row.
3. How does this toy problem relate to image classification?
The principle is pretty much the same. Instead of a tensor of size n\*1, we have a tensor with 28\*28 entries. The network learns weights by extracting features in our dataset, in order to minimise the loss and correctly classify the inputs.

Write short answers (2–3 lines each).


## 10) Bridge to Computer Vision

So far:
- inputs = vectors of size 3
- layers = fully-connected

Next session:
- inputs = images `(B, C, H, W)`
- layers = convolutions
- same training logic

👉 **Architecture changes, learning principles stay the same.**


## Part II — Training on MNIST

Check the next notebook